In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import scipy as sp

# Loading and splitting the dataset

In [ ]:
img_size = 224
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Resize((img_size, img_size))])

trainset = torchvision.datasets.Flowers102(root='./data', split='train', download=True, transform=transform)
valset = torchvision.datasets.Flowers102(root='./data', split='val', download=True, transform=transform)
testset = torchvision.datasets.Flowers102(root='./data', split='test', download=True, transform=transform)

In [ ]:
trainset

Dataset Flowers102
    Number of datapoints: 1020
    Root location: ./data
    split=train
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
           )

In [ ]:
valset

Dataset Flowers102
    Number of datapoints: 1020
    Root location: ./data
    split=val
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
           )

In [ ]:
testset

Dataset Flowers102
    Number of datapoints: 6149
    Root location: ./data
    split=test
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
           )

# Data preparation

In [ ]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [ ]:
batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

# Building the model

In [15]:
class FlowerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv0 = nn.Conv2d(3, 64, kernel_size=(3,3), stride=1, padding=1)
        self.bn0 = nn.BatchNorm2d(64)
        self.act0 = nn.ReLU()
        # self.drop0 = nn.Dropout(0.1)

        self.conv1 = nn.Conv2d(64, 64, kernel_size=(3,3), stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.act1 = nn.ReLU()
        # self.drop1 = nn.Dropout(0.1)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3), stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        # self.flat = nn.Flatten()
        # self.avg = nn.AvgPool2d(1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Adaptive average pooling to (1, 1)


        self.fc3 = nn.Linear(128, 512)
        self.act3 = nn.ReLU()
        # self.drop3 = nn.Dropout(0.25)

        self.fc4 = nn.Linear(512, 102)

    def forward(self, x):
        # input 3x224x224, output 64x224x224
        x = self.act0(self.bn0(self.conv0(x)))
        # x = self.drop0(x)
        # input 64x224x224, output 64x224x224
        x = self.act1(self.bn1(self.conv1(x)))
        # x = self.drop1(x)
        # input 64x224x224, output 128x224x224
        x = self.act2(self.bn2(self.conv2(x)))
        # input 128x224x224, output 128x112x112
        x = self.pool2(x)
        # input 128x112x112, output
        # x = self.flat(x)
        # x = self.avg(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        # input , output 512
        x = self.act3(self.fc3(x))
        # x = self.drop3(x)
        # input 512, output 102
        x = self.fc4(x)
        return x


In [ ]:
model = FlowerModel()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
early_stopping_patience = 10
best_val_accuracy = float(0)
patience_counter = 0

In [ ]:
model.to(dev)

FlowerModel(
  (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU()
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc3): Linear(in_features=128, out_features=512, bias=True)
  (act3): ReLU()
  (fc4): Linear(in_features=512, out_features=102, bias=True)
)

In [ ]:
from torchsummary import summary

summary(model, (3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
            Conv2d-7        [-1, 128, 224, 224]          73,856
       BatchNorm2d-8        [-1, 128, 224, 224]             256
              ReLU-9        [-1, 128, 224, 224]               0
        MaxPool2d-10        [-1, 128, 112, 112]               0
AdaptiveAvgPool2d-11            [-1, 128, 1, 1]               0
           Linear-12                  [-1, 512]          66,048
             ReLU-13                  [-1, 512]               0
           Linear-14                  [

# Training and evaluating it

In [13]:
n_epochs = 100
for epoch in range(n_epochs):
    for inputs, labels in trainloader:
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # forward, backward, and then weight update
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()

    acc = 0
    count = 0
    for inputs, labels in valloader:
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        y_pred = model(inputs)
        acc += (torch.argmax(y_pred, 1) == labels).float().sum()
        count += len(labels)
    acc /= count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

    if acc > best_val_accuracy:
        best_val_accuracy = acc
        patience_counter = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered")
            break

model.load_state_dict(torch.load('best_model.pth'))
torch.save(model.state_dict(torch.load('best_model.pth')), "flowermodel.pth")

Epoch 0: model accuracy 7.55%
Epoch 1: model accuracy 10.49%
Epoch 2: model accuracy 12.55%
Epoch 3: model accuracy 16.47%
Epoch 4: model accuracy 16.47%
Epoch 5: model accuracy 19.12%
Epoch 6: model accuracy 19.51%
Epoch 7: model accuracy 20.69%
Epoch 8: model accuracy 21.37%
Epoch 9: model accuracy 21.57%
Epoch 10: model accuracy 24.41%
Epoch 11: model accuracy 24.31%
Epoch 12: model accuracy 23.53%
Epoch 13: model accuracy 23.92%
Epoch 14: model accuracy 25.39%
Epoch 15: model accuracy 25.10%
Epoch 16: model accuracy 25.00%
Epoch 17: model accuracy 25.00%
Epoch 18: model accuracy 25.29%
Epoch 19: model accuracy 25.39%
Epoch 20: model accuracy 25.10%
Epoch 21: model accuracy 25.49%
Epoch 22: model accuracy 25.78%
Epoch 23: model accuracy 25.10%
Epoch 24: model accuracy 25.59%
Epoch 25: model accuracy 25.49%
Epoch 26: model accuracy 25.00%
Epoch 27: model accuracy 25.29%
Epoch 28: model accuracy 25.59%
Epoch 29: model accuracy 26.47%
Epoch 30: model accuracy 25.20%
Epoch 31: model acc

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1898: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [14]:
acc = 0
count = 0
for inputs, labels in testloader:
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    y_pred = model(inputs)
    acc += (torch.argmax(y_pred, 1) == labels).float().sum()
    count += len(labels)
acc /= count
print("model accuracy %.2f%%" % (acc*100))

model accuracy 21.76%


In [ ]:
# lr      test score    trained more  dropout 0.75  dropout 0.25   best with dropout     without dropout

# 0.001   15.4%         15.16%        10.49%        17.53%         20.15%                21.76%
# 0.01    6.47%
# 0.0001  6.98%
# 0.0006  14.15%